In [1]:
import os
import wikipedia
import psycopg2
import openai
from dotenv import load_dotenv, find_dotenv
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional
from langchain.tools import Tool
from langchain.utilities import SerpAPIWrapper

# Load environment variables
load_dotenv()

_ = load_dotenv(find_dotenv('src/.env'))  # Read local .env file

openai.api_key = os.environ["OPENAI_API_KEY"]
DATABASE_URL = os.environ["DATABASE_URL"]
SERPAPI_KEY = os.environ["SERPAPI_KEY"]  # For Google search

In [2]:
import os
import wikipedia
import psycopg2
import openai
from dotenv import load_dotenv, find_dotenv
import uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional
from langchain.tools import Tool
from langchain.utilities import SerpAPIWrapper

# Load environment variables
load_dotenv()
_ = load_dotenv(find_dotenv())  # Read local .env file

openai.api_key = os.environ["OPENAI_API_KEY"]
DATABASE_URL = os.environ["DATABASE_URL"]
SERPAPI_KEY = os.environ["SERPAPI_KEY"]  # For Google search

# Initialize FastAPI
app = FastAPI()

# Database connection
def get_db_connection():
    return psycopg2.connect(DATABASE_URL)

# Input model
class QueryRequest(BaseModel):
    query: str
    location: Optional[str] = None  # For queries like "near me"

# Dictionary to store chat history
global chat_history
chat_history = []

# Function to maintain chat history
def update_chat_history(user_query: str, bot_response: str):
    global chat_history
    
    # Reset chat history if conversation goes off-topic
    if classify_query_with_llm(user_query) == "Irrelevant":
        chat_history = []

    chat_history.append({"user": user_query, "bot": bot_response})

# Function to generate SQL query using LLM
def generate_sql_query(user_query: str):
    prompt = f"""
    You are an AI trained to generate SQL queries for a PostgreSQL database based on user input.
    The database has a table named `restaurant_menu` with the following columns:
    - `restaurant_name` (VARCHAR)
    - `menu_category` (VARCHAR)
    - `item_id` (INTEGER)
    - `menu_item` (VARCHAR)
    - `menu_description` (TEXT)
    - `ingredient_name` (VARCHAR)
    - `confidence` (FLOAT)
    - `categories` (VARCHAR)
    - `address1` (TEXT)
    - `city` (VARCHAR)
    - `zip_code` (VARCHAR)
    - `country` (VARCHAR)
    - `state` (VARCHAR)
    - `rating` (VARCHAR)
    - `review_count` (VARCHAR)
    - `price` (VARCHAR)

    Convert the following user query into a valid SQL query for the `restaurant_menu` table:
    "{user_query}"

    Only return the SQL query, no extra explanations, and do not include any code formatting markers.
    Remember review_count, rating, and price columns are varchar. Perform cast conversion where necessary.
    Don't forget to use DISTINCT as there are many duplicate restaurant names, ingredients, menu items etc.
    Everything in the database is lower case, so remember to lowercase query values.
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}],
        max_tokens=100,
    )
    
    return response["choices"][0]["message"]["content"].strip()

# Function to execute SQL query
def execute_sql_query(sql: str):
    conn = get_db_connection()
    cursor = conn.cursor()
    try:
        cursor.execute(sql)
        results = cursor.fetchall()
        if not results:
            return "No matching results found."
        formatted_results = "\n".join([", ".join(map(str, row)) for row in results])
        return formatted_results
    except Exception as e:
        return f"Error executing query: {e}"
    finally:
        cursor.close()
        conn.close()

# Function to query restaurant database
def query_restaurant_db(query: str, location: Optional[str]):
    sql_query = generate_sql_query(query)
    return execute_sql_query(sql_query)

# Wikipedia search function
def search_wikipedia(query: str):
    try:
        return wikipedia.summary(query, sentences=2)
    except wikipedia.exceptions.PageError:
        return "No relevant Wikipedia article found."
    except wikipedia.exceptions.DisambiguationError as e:
        return f"Did you mean: {', '.join(e.options[:5])}?"

# Google Search using SerpAPI
def search_google(query: str):
    google_search = SerpAPIWrapper(serpapi_api_key=SERPAPI_KEY)
    return google_search.run(query)

# LLM response function
def generate_response(prompt: str):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a helpful food assistant."},
                  {"role": "user", "content": prompt}]
    )
    return response["choices"][0]["message"]["content"]

# Helper function to classify query using LLM
def classify_query_with_llm(query: str) -> str:
    prompt = f"""
    Classify the following query into one of the categories: Greeting, SQL, Wikipedia, GoogleSearch, Irrelevant.
    
    Query: {query}
    Categories:
    - Greeting: Greetings like "Hello", "Hi", "Hey", "How are you?", "Good morning", "Good evening", "What can you do".
    - SQL: Queries that involve database searches, like finding restaurants, prices, or ratings.
    - Wikipedia: Queries related to food items like sushi, food ingredients, cuisine history, nutrition, or similar background information.
    - GoogleSearch: Queries asking for trending, popular, or recent food-related information, such as reviews or new restaurants.
    - Irrelevant: If the query is about politics, violence, technology, sports, general knowledge, or anything unrelated to food or restaurants.
    
    Return only one of the categories: "Greeting", "SQL", "Wikipedia", "GoogleSearch", "Irrelevant" as string. Only the words.
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}],
        max_tokens=20,
    )
    return response["choices"][0]["message"]["content"].strip()

# Helper function: return recent chat history as a string.
# It takes the last two Q/A pairs and trims the string if it exceeds max_length.
def get_recent_history(max_entries=2, max_length=250) -> str:
    recent_entries = chat_history[-max_entries:]
    history_str = "\n".join([f"User: {entry['user']}\nBot: {entry['bot']}" for entry in recent_entries])
    if len(history_str) > max_length:
        history_str = history_str[-max_length:]
    return history_str

@app.post("/chat/")
def chatbot(request: QueryRequest):
    query = request.query.lower()
    query_type = classify_query_with_llm(query)
    recent_history = get_recent_history()  # Recent two interactions, trimmed

    if query_type == "Greeting":
        bot_response = "Hello! I’m your restaurant and food assistant. You can ask me about restaurants, menus, food history, or reviews!"
    elif query_type == "SQL":
        db_response = query_restaurant_db(query, request.location)
        full_prompt = (f"Chat history:\n{recent_history}\n"
                       f"User asked: {query}.\nDatabase says:\n{db_response}")
        bot_response = generate_response(full_prompt)
    elif query_type == "Wikipedia":
        # Combine recent history with current query, then trim to a safe limit (e.g. 200 chars)
        combined_query = f"{recent_history} {query}"
        if len(combined_query) > 300:
            wiki_query = combined_query[-300:]
        else:
            wiki_query = combined_query
        wiki_info = search_wikipedia(wiki_query)
        full_prompt = (f"Chat history:\n{recent_history}\n"
                       f"User asked: {query}.\nWikipedia info:\n{wiki_info}")
        bot_response = generate_response(full_prompt)
    elif query_type == "GoogleSearch":
        combined_query = f"{recent_history} {query}"
        if len(combined_query) > 300:
            web_query = combined_query[-300:]
        else:
            web_query = combined_query
        web_info = search_google(web_query)
        full_prompt = (f"Chat history:\n{recent_history}\n"
                       f"User asked: {query}.\nWeb search results:\n{web_info}")
        bot_response = generate_response(full_prompt)
    else:
        bot_response = "I can only answer food-related or restaurant queries."
    
    update_chat_history(query, bot_response)
    
    return {"response": bot_response}

'''
# Run the API on localhost
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)
'''

'\n# Run the API on localhost\nif __name__ == "__main__":\n    uvicorn.run(app, host="127.0.0.1", port=8000)\n'

In [6]:
# Test generate_sql_query function
user_query = "Find the number of best restaurants in San Francisco with a rating greater than 4.5"
sql_query = generate_sql_query(user_query)
print(sql_query)


SELECT COUNT(DISTINCT restaurant_name) 
FROM restaurant_menu 
WHERE city = 'san francisco' 
AND rating::FLOAT > 4.5;


In [7]:
response = execute_sql_query(sql_query)
print(response)

21


In [8]:
# Test generate_sql_query function
user_query = "List all the restaurants in the database"
sql_query = generate_sql_query(user_query)
print(sql_query)


SELECT DISTINCT restaurant_name FROM restaurant_menu;


In [ ]:
response = execute_sql_query(sql_query)
print(response)

In [ ]:
# Test search_wikipedia function
wiki_query = "Italian cuisine"
wiki_response = search_wikipedia(wiki_query)
print(wiki_response)

In [ ]:
# Test generate_response function
prompt = "What are the benefits of a vegan diet?"
response = generate_response(prompt)
print(response)

In [ ]:
# Simulate FastAPI POST request for chatbot function
from pydantic import BaseModel
from typing import Optional

# Recreate the QueryRequest model (if not already imported)
class QueryRequest(BaseModel):
    query: str
    location: Optional[str] = None
    session_id: Optional[str] = None  # Make session_id optional

In [10]:
# Test the chatbot function
request = QueryRequest(query="Find the number of best restaurants in San Francisco with a rating greater than 4")
chatbot_response = chatbot(request)
print(chatbot_response)

{'response': 'I found that there are 94 best restaurants in San Francisco with a rating greater than 4. Would you like more information on any specific restaurant or cuisine in San Francisco?'}


In [ ]:
# Test the chatbot function
request = QueryRequest(query="Can you tell me about Italian Cuisine?")
chatbot_response = chatbot(request)
print(chatbot_response)

In [ ]:
# Test the chatbot function
request = QueryRequest(query="What is the history of pizza, and which restaurants in my area are known for it?")
chatbot_response = chatbot(request)
print(chatbot_response)

In [ ]:
request = QueryRequest(query="Can you tell me its ingredients?")
chatbot_response = chatbot(request)
print(chatbot_response)

In [ ]:
print(chat_history)

In [ ]:
request = QueryRequest(query="What are the best restaurants selling that in Boston?")
chatbot_response = chatbot(request)
print(chatbot_response)

In [ ]:
import os
import wikipedia
import psycopg2
import openai
from dotenv import load_dotenv, find_dotenv
import uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional
from langchain.tools import Tool
from langchain.utilities import SerpAPIWrapper

# Load environment variables
load_dotenv()
_ = load_dotenv(find_dotenv())  # Read local .env file

openai.api_key = os.environ["OPENAI_API_KEY"]
DATABASE_URL = os.environ["DATABASE_URL"]
SERPAPI_KEY = os.environ["SERPAPI_KEY"]  # For Google search

# Initialize FastAPI
app = FastAPI()

# Database connection
def get_db_connection():
    return psycopg2.connect(DATABASE_URL)

# Input model
class QueryRequest(BaseModel):
    query: str
    location: Optional[str] = None  # For queries like "near me"

# Dictionary to store chat history
global chat_history
chat_history = []

# Function to maintain chat history
def update_chat_history(user_query: str, bot_response: str):
    global chat_history
    
    # Reset chat history if conversation goes off-topic
    if classify_query_with_llm(user_query) == "Irrelevant":
        chat_history = []

    chat_history.append({"user": user_query, "bot": bot_response})

# Function to generate SQL query using LLM
def generate_sql_query(user_query: str):
    prompt = f"""
    You are an AI trained to generate SQL queries for a PostgreSQL database based on user input.
    The database has a table named `restaurant_menu` with the following columns:
    - `restaurant_name` (VARCHAR)
    - `menu_category` (VARCHAR)
    - `item_id` (INTEGER)
    - `menu_item` (VARCHAR)
    - `menu_description` (TEXT)
    - `ingredient_name` (VARCHAR)
    - `confidence` (FLOAT)
    - `categories` (VARCHAR)
    - `address1` (TEXT)
    - `city` (VARCHAR)
    - `zip_code` (VARCHAR)
    - `country` (VARCHAR)
    - `state` (VARCHAR)
    - `rating` (VARCHAR)
    - `review_count` (VARCHAR)
    - `price` (VARCHAR)

    Convert the following user query into a valid SQL query for the `restaurant_menu` table:
    "{user_query}"

    Only return the SQL query, no extra explanations, and do not include any code formatting markers.
    Remember review_count, rating, and price columns are varchar. Perform cast conversion where necessary.
    Don't forget to use DISTINCT as there are many duplicate restaurant names, ingredients, menu items etc.
    Everything in the database is lower case, so remember to lowercase query values.
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}],
        max_tokens=100,
    )
    
    return response["choices"][0]["message"]["content"].strip()

# Function to execute SQL query
def execute_sql_query(sql: str):
    conn = get_db_connection()
    cursor = conn.cursor()
    try:
        cursor.execute(sql)
        results = cursor.fetchall()
        if not results:
            return "No matching results found."
        formatted_results = "\n".join([", ".join(map(str, row)) for row in results])
        return formatted_results
    except Exception as e:
        return f"Error executing query: {e}"
    finally:
        cursor.close()
        conn.close()

# Function to query restaurant database
def query_restaurant_db(query: str, location: Optional[str]):
    sql_query = generate_sql_query(query)
    return execute_sql_query(sql_query)

# Wikipedia search function
def search_wikipedia(query: str):
    try:
        return wikipedia.summary(query, sentences=2)
    except wikipedia.exceptions.PageError:
        return "No relevant Wikipedia article found."
    except wikipedia.exceptions.DisambiguationError as e:
        return f"Did you mean: {', '.join(e.options[:5])}?"

# Google Search using SerpAPI
def search_google(query: str):
    google_search = SerpAPIWrapper(serpapi_api_key=SERPAPI_KEY)
    return google_search.run(query)

# LLM response function
def generate_response(prompt: str):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a helpful food assistant."},
                  {"role": "user", "content": prompt}]
    )
    return response["choices"][0]["message"]["content"]

# Helper function to classify query using LLM
def classify_query_with_llm(query: str) -> str:
    prompt = f"""
    Classify the following query into one of the categories: Greeting, SQL, Wikipedia, GoogleSearch, Irrelevant.
    
    Query: {query}
    Categories:
    - Greeting: Greetings like "Hello", "Hi", "Hey", "How are you?", "Good morning", "Good evening", "What can you do".
    - SQL: Queries that involve database searches, like finding restaurants, prices, or ratings.
    - Wikipedia: Queries related to food items like sushi, food ingredients, cuisine history, nutrition, or similar background information.
    - GoogleSearch: Queries asking for trending, popular, or recent food-related information, such as reviews or new restaurants.
    - Irrelevant: If the query is about politics, violence, technology, sports, general knowledge, or anything unrelated to food or restaurants.
    
    Return only one of the categories: "Greeting", "SQL", "Wikipedia", "GoogleSearch", "Irrelevant" as string. Only the words.
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}],
        max_tokens=20,
    )
    return response["choices"][0]["message"]["content"].strip()


@app.post("/chat/")
def chatbot(request: QueryRequest):
    query = request.query.lower()
    query_type = classify_query_with_llm(query)
    
    if query_type == "Greeting":
        bot_response = "Hello! I’m your restaurant and food assistant. You can ask me about restaurants, menus, food history, or reviews!"
    elif query_type == "SQL":
        db_response = query_restaurant_db(query, request.location)
        bot_response = generate_response(f"User asked: {query}. Database says:\n{db_response}")
    elif query_type == "Wikipedia":
        wiki_info = search_wikipedia(query)
        bot_response = generate_response(f"User asked: {query}. Wikipedia info:\n{wiki_info}")
    elif query_type == "GoogleSearch":
        web_info = search_google(query)
        bot_response = generate_response(f"User asked: {query}. Web search results:\n{web_info}")
    else:
        bot_response = "I can only answer food-related or restaurant queries."
    
    update_chat_history(query, bot_response)
    
    return {"response": bot_response}
'''
# Run the API on localhost
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)
'''